In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.lines as mlines
import os
import matplotlib as mpl
from PIL import Image
from io import BytesIO
mpl.rc('figure', dpi=400, figsize=(10,10))
mpl.rc('savefig', dpi=400)
import numpy as np

from math import log10, floor

def round_sig(x, sig=1):
    """Round a value to N sig fig.

    Parameters
    ----------
    x : float
        Value to round
    sig : int, optional
        Number of sig figs, default is 1

    Returns
    -------
    float
        Rounded value

    """
    return round(x, sig - int(floor(log10(abs(x)))) - 1)

In [0]:
project_id = 'hypothermia-bayescmd'
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

In [0]:
LWP479_query = """
SELECT
    CBFn,
    K_sigma,
    phi,
    n_h,
    k_aut,
    R_auto,
    Xtot_n,
    Q_10,
    NADpool,
    O2_n,
    r_n,
    r_0,
    r_t,
    NRMSE,
    "LWP479" as model_name
FROM
  LWP479.bph1_1
ORDER BY
  NRMSE ASC
LIMIT
  50000
"""

In [0]:
LWP475_query = """
SELECT
    CBFn,
    K_sigma,
    phi,
    n_h,
    k_aut,
    R_auto,
    Xtot_n,
    Q_10,
    NADpool,
    O2_n,
    r_n,
    r_0,
    r_t,
    NRMSE,
    "LWP475" as model_name
 FROM 
  LWP475.bph1
 ORDER BY NRMSE ASC 
 LIMIT 50000
 """

In [0]:
df475 = client.query(LWP475_query).to_dataframe()

In [0]:
df479 = client.query(LWP479_query).to_dataframe()

In [0]:
df = pd.concat([df475, df479])
df.head()

,CBFn,K_sigma,phi,n_h,k_aut,R_auto,Xtot_n,Q_10,NADpool,O2_n,r_n,r_0,r_t,NRMSE,model_name
0,0.006447,10.067759,0.041978,2.760951,0.927100,0.032600,5.633674,5.886396,3.949605,0.020345,0.015919,0.012346,0.019015,0.677611,LWP475
1,0.006894,8.211408,0.042979,2.704179,0.812034,0.275974,5.041412,5.958105,2.741552,0.019445,0.017545,0.011756,0.017131,0.680020,LWP475
2,0.006581,10.789513,0.041824,2.486855,0.941600,0.069301,5.580235,5.852527,3.523267,0.020557,0.020515,0.014589,0.019085,0.683028,LWP475
3,0.006613,9.650557,0.041817,2.896765,0.967578,0.493388,5.203070,5.973608,2.707903,0.019477,0.015616,0.015046,0.019686,0.684943,LWP475
4,0.006599,10.079713,0.043052,2.605980,0.824446,0.240908,5.332797,5.971031,3.939768,0.021628,0.016338,0.012433,0.017573,0.688737,LWP475


In [0]:
priors =  {
        "CBFn": [
            "uniform",
            [
                0.0064,
                0.0096
            ]
        ],
        "K_sigma": [
            "uniform",
            [
                8.0,
                12.0
            ]
        ],
        "phi": [
            "uniform",
            [
                0.0288,
                0.0432
            ]
        ],
        "n_h": [
            "uniform",
            [
                2.0,
                3.0
            ]
        ],
        "k_aut": [
            "uniform",
            [
                0.0,
                1.0
            ]
        ],
        "R_auto": [
            "uniform",
            [
                0.0,
                10.0
            ]
        ],
        "Xtot_n": [
            "uniform",
            [
                3.5,
                8.0
            ]
        ],
        "Q_10": [
            "uniform",
            [
                0.001,
                6.0
            ]
        ],
        "NADpool": [
            "uniform",
            [
                1.5,
                4.5
            ]
        ],
        "O2_n": [
            "uniform",
            [
                0.0192,
                0.0288
            ]
        ],
        "r_n": [
            "uniform",
            [
                0.01496,
                0.02244
            ]
        ],
        "r_0": [
            "uniform",
            [
                0.01008,
                0.015119999999999998
            ]
        ],
        "r_t": [
            "uniform",
            [
                0.0144,
                0.0216
            ]
        ]
    }

In [0]:
def plot_comparison_diag_medians(g, df, medians):

    for i, var in enumerate(g.x_vars):
        ax = g.axes[i][i]
        LWP475_median = np.median(df[var][df['model_name']=='LWP475'])
        ax.axvline(LWP475_median, color=sns.color_palette()[0], linewidth=2)
        
        LWP479_median = np.median(df[var][df['model_name']=='LWP479'])
        ax.axvline(LWP479_median, color=sns.color_palette()[1], linewidth=2)
        
        
        ax.text(
            0.05,
            1.45,
            "LWP475: {:.3g}\nLWP479: {:.3g}".format(LWP475_median,
                                                    LWP479_median),
            verticalalignment='center',
            transform=ax.transAxes,
            fontsize=8)
        


    return True

In [0]:
def medians_comparison_kde_plot(x, y, medians, **kws):
    """Plot bivariate KDE with median of distribution marked on,
    comparing between groups.

    Parameters
    ----------
    x : array-like
        Array-like of data to plot.
    y : array-like
        Array-like of data to plot.
    medians : :obj:`dict`
        Dictionary of parameter, median pairings.
    kws : key, value pairings.
        Other keyword arguments to pass to :obj:`sns.distplot`.

    Returns
    -------
    ax : :obj:`matplotlib.AxesSubplot`
        AxesSubplot object of univariate KDE and bar plot with median marked
        on as well as text.

    """
    ax = plt.gca()
    ax = sns.kdeplot(x, y, ax=ax, **kws)
    # color = infer_from_cmap(kws['cmap'])
    x_median = x.median()
    y_median = y.median()
    ax.plot(x_median, y_median, 'X', markerfacecolor=kws['color'],
            markeredgecolor='k', markeredgewidth=1.5, markersize=6)
    return ax

In [0]:
parameters= list(priors.keys())
medians={}
with sns.plotting_context("paper", rc={"xtick.labelsize": 12,
                                       "ytick.labelsize": 12,
                                       "axes.labelsize": 12}):   
    g = sns.PairGrid(df,
                     vars=parameters,
                     diag_sharey=False,
                     height=0.5,
                     hue='model_name')
    
    g.map_diag(sns.distplot, hist_kws=dict(alpha=0.5))
    plot_comparison_diag_medians(g, df, medians=medians)
    g.map_lower(medians_comparison_kde_plot, medians=medians)
    n_ticks=4
    for i, j in zip(*np.triu_indices_from(g.axes, 1)):
        g.axes[i, j].set_visible(False)
    for ii, ax in enumerate(g.axes.flat):
        for label in ax.get_xticklabels():
            label.set_rotation(75)
        ax.xaxis.labelpad=5
        ax.yaxis.labelpad=5
        ii_y = ii // len(parameters)
        ii_x = ii % len(parameters)
        ax.set_ylim(priors[parameters[ii_y]][1])
        ax.set_xlim(priors[parameters[ii_x]][1])
        xmax = priors[parameters[ii_x]][1][1]
        xmin = priors[parameters[ii_x]][1][0]
        xticks = np.arange(xmin, xmax,
                           round_sig((xmax - xmin) / n_ticks, sig=1))
        ax.set_xticks(xticks)
        ax.set_xlabel(ax.get_xlabel(), labelpad=1, rotation=30, fontsize=8)
        ax.set_ylabel(ax.get_ylabel(), labelpad=15, rotation=45, fontsize=8)
    lines = []
    # lines.append(('True Value', mlines.Line2D([], [], color='black')))
    lines.append(('Posterior Median - LWP475', mlines.Line2D([], [], color=sns.color_palette()[0])))
    lines.append(('Posterior Median - LWP479', mlines.Line2D([], [], color=sns.color_palette()[1])))
    g.set(yticklabels=[])
    g.set(xticklabels=[])
    g.fig.legend(labels=[l[0] for l in lines],
                 handles=[l[1] for l in lines],
                 bbox_to_anchor=(0.35, 0.95), loc=2, prop={"size": 12})

    # g.fig.tight_layout()
    g.fig.subplots_adjust(wspace=0.15, hspace=0.25)

In [0]:
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
g.savefig('/content/gdrive/My Drive/Colab Notebooks/comparison_posteriors_bph1.png')

In [0]:
g = sns.FacetGrid(df, hue='model_name', height=3)
g = (g.map(sns.distplot, 'Q_10'))

In [0]:
g.savefig('/content/gdrive/My Drive/Colab Notebooks/Q_10_marginal_bph1.png')